# Using the MODA framework to analyze the SF 311 data

Source of data: https://data.sfgov.org/City-Infrastructure/311-Cases/vw6y-z8j6/data

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import numpy as np

## The time window to bucket samples
TIME_RANGE = '4H'

## File path (original data is ~1GB, this is a reduced version with only categories and dates)
#DATAPATH = "SF-311-categories-2018.csv"
DATAPATH = "SF311_simplified.csv"

In [7]:
#Raw data sample:
raw = pd.read_csv(DATAPATH,nrows=100)
raw.head(10)

,Unnamed: 0,Opened,Category,Request Details
0,0,07/02/2008 03:16:55 PM,Tree Maintenance,Lifted_sidewalk_tree_roots
1,1,07/02/2008 04:47:08 PM,Tree Maintenance,Hanging_limb
2,2,07/02/2008 05:54:07 PM,Tree Maintenance,Blocking_street_lights
3,3,07/03/2008 12:38:04 PM,Tree Maintenance,Sprinkler_system_issues
4,4,07/03/2008 12:44:26 PM,Street and Sidewalk Cleaning,Other Loose Garbage
5,5,07/03/2008 01:24:07 PM,Tree Maintenance,Lifted_sidewalk_tree_roots
6,6,07/07/2008 03:14:07 PM,Tree Maintenance,Near_communication_line
7,7,07/08/2008 06:25:18 PM,Tree Maintenance,About_to_fall
8,8,07/09/2008 02:57:52 PM,Tree Maintenance,Near_communication_line
9,9,07/09/2008 04:57:24 PM,Tree Maintenance,Near_communication_line


In [8]:
from moda.dataprep.raw_to_ts import raw_to_ts
from moda.dataprep.ts_to_range import ts_to_range

def prep_data(datapath, time_range='24H', nrows=None, min_date=None, max_date=None, save_files=False,file_prefix = "", usecols=None):
    """
    Takes a raw data with timestamps (date column), categories (category column) and additional columns,
    and turns it into a ranged time-series: Group the original raw data by time interval (time_range) and category.
    Result is the number of samples per category in each time range.
    :param datapath: the path to the csv file
    :param time_range: the time_range according to which the data is grouped by
    :param nrows: limits the number of rows read from the csv
    :param min_date: filters out ranges prior to min_date
    :param max_date: filters out ranges after max_date
    :param save_files: Whether to save intermediate csvs
    :returns a pd.DataFrame with one value per time_range and category.
    This value is the number of samples within this range for a specific category
    """

    if nrows is None:
        raw = pd.read_csv(datapath, usecols=usecols)
    else:
        raw = pd.read_csv(datapath, usecols=usecols, nrows=nrows)

    raw = raw.rename(columns={'Opened': 'date', 'Category': 'category'})

    # Create a time series dataframe
    ts = raw_to_ts(raw, min_date=min_date, max_date=max_date)

    # Divide time series to ranges and categories
    ranged_ts = ts_to_range(ts, time_range=time_range)

    if save_files:
        if nrows is None:
            ts.to_csv("{}_ts.csv".format(file_prefix))
            ranged_ts.to_csv("{0}_ranged_ts_{1}.csv".format(file_prefix,time_range))
        else:
            ts.to_csv("{0}_ts_{1}_rows.csv".format(file_postfix,nrows))
            ranged_ts.to_csv("{0}_ranged_ts_{1}_{2}_rows.csv".format(file_prefix,time_range,nrows))
    return ranged_ts



In [3]:
# This might take some time to load
ranged_ts_24H_2018 = prep_data(DATAPATH,time_range='24H',min_date="01-01-2018",save_files = True,file_prefix = "SF")

C:\ProgramData\Anaconda3\lib\site-packages\moda\dataprep\ts_to_range.py:17: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  range_grp = ts.groupby([pd.TimeGrouper(time_range), 'category']).agg('count')


In [9]:
# This might take some time to load
ranged_ts_4H_2016_2018 = prep_data(DATAPATH,time_range='4H',min_date="01-01-2016",save_files = True,file_prefix = "SF")

C:\ProgramData\Anaconda3\lib\site-packages\moda\dataprep\ts_to_range.py:17: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  range_grp = ts.groupby([pd.TimeGrouper(time_range), 'category']).agg('count')


Now that we have a file with the number of events per category per time, we can start modeling.

First, in order to be able to estimate our models, we use [TagAnomaly](https://github.com/Microsoft/TagAnomaly) to tag the points we think are showing trends in the data. Taganomaly can be found here: https://github.com/Microsoft/TagAnomaly
Second, we join the tagged dataset with the time series dataset. Each sample which isn't included in the tagged dataset is assumed to be non-trending (or normal)


In [6]:
labels24H = pd.read_csv('SF_24H_anomalies_only.csv',usecols=['date','category','value'])
labels24H.date = pd.to_datetime(labels24H.date)
labels24H.head()


,date,category,value
0,2018-04-17,Blocked Street or SideWalk,36
1,2018-04-18,Blocked Street or SideWalk,40
2,2018-04-19,Blocked Street or SideWalk,31
3,2018-04-20,Blocked Street or SideWalk,38
4,2018-04-17,Blocked Street or SideWalk,36


In [7]:
labels1H = pd.read_csv('SF_1H_anomalies_only.csv',usecols=['date','category','value'])
labels1H.date = pd.to_datetime(labels1H.date)
labels1H.head()

,date,category,value
0,2018-01-30 08:00:00,Abandoned Vehicle,16
1,2018-01-30 09:00:00,Abandoned Vehicle,21
2,2018-02-01 17:00:00,Abandoned Vehicle,18
3,2018-02-05 08:00:00,Abandoned Vehicle,18
4,2018-02-12 15:00:00,Abandoned Vehicle,16


In [ ]:
ranged_ts_24H_2018.reset_index().head()

In [ ]:
ranged_ts_1H_2018.reset_index().head()

In [ ]:
df24H = pd.merge(ranged_ts_24H_2018.reset_index(),labels24H,how='left',on=['date','category'])
df24H['label'] = np.where(np.isnan(df24H['value_y']),0,1)
df24H = df24H.drop(columns = 'value_y').rename(columns = {'value_x':'value'})
df24H.to_csv("SF24H_labeled.csv")

df1H = pd.merge(ranged_ts_1H_2018.reset_index(),labels1H,how='left',on=['date','category'])
df1H['label'] = np.where(np.isnan(df1H['value_y']),0,1)
df1H = df1H.drop(columns = 'value_y').rename(columns = {'value_x':'value'})
df1H.to_csv("SF24H_labeled.csv")

All examples:

In [ ]:
df24H.head()

Positive examples:

In [ ]:
df24H[df24H.label > 0].head()

Evaluating different models

In [ ]:
from moda.evaluators.eval import get_evaluation_metrics, get_final_metrics, eval_models
from moda.models.azure_anomaly_detection.azure_ad import AzureAnomalyTrendinessDetector
from moda.models.data_reader import read_data
from moda.models.ma_seasonal.ma_seasonal_model import MovingAverageSeasonalTrendinessDetector
from moda.models.stl.stl_model import STLTrendinessDetector
from moda.models.twitter.anomaly_detect_multicategory import TwitterAnomalyTrendinessDetector


def run_model(datapath, freq, min_date='01-01-2018', plot=True, model_name='stl', min_value=10):
    print("Loading file {0}, with frequency {1}. Model name = {2}".format(datapath, freq, model_name))
    dataset = read_data(datapath, min_date=min_date)
    dataset = dataset.rename(columns={'is_anomaly': 'label'})

    if len(dataset.index.levels) > 1:
        categories = dataset.index.levels[1]
        print("categories found = {}".format(categories))

    if model_name == 'twitter':
        model = TwitterAnomalyTrendinessDetector(is_multicategory=True, freq=freq, min_value=min_value, threshold=None,
                                                 max_anoms=0.49, seasonality_freq=7)

    if model_name == 'ma_seasonal':
        model = MovingAverageSeasonalTrendinessDetector(is_multicategory=True, freq=freq, min_value=min_value,
                                                        anomaly_type='or',
                                                        num_of_std=3)

    if model_name == 'stl':
        model = STLTrendinessDetector(is_multicategory=True, freq=freq, min_value=min_value,
                                      anomaly_type='or',
                                      num_of_std=2.5, lo_delta=0)

    if model_name == 'azure':
        dirname = os.path.dirname(__file__)
        filename = os.path.join(dirname, 'config/config.json')
        subscription_key = get_azure_subscription_key(filename)
        model = AzureAnomalyTrendinessDetector(is_multicategory=True, freq=freq, min_value=min_value,
                                               subscription_key=subscription_key)

    prediction = model.predict(dataset, verbose=True)
    raw_metrics = get_evaluation_metrics(dataset[['value']], prediction[['prediction']], dataset[['label']],
                                         window_size_for_metrics=5)
    metrics = get_final_metrics(raw_metrics)
    print(metrics)

    ## Plot each category
    if plot:
        _, file = os.path.split(datapath)
        print("Plotting...")
        model.plot(labels=dataset['label'], postfix=file)

    return prediction

In [ ]:
prediction = run_model("SF24H_labeled.csv",freq='24H',model_name='stl')